ZOMNA DE IMPORT

 ºPara poder usar pyspark en VScode tenemos que usar findspark.init()

In [2]:
import findspark
findspark.init()

In [4]:
from pyspark.sql.functions import expr, col, udf, array
from pyspark.sql.types import StructField, StructType, StringType, DecimalType, LongType, FloatType
from pyspark.sql import SparkSession

In [5]:


spark = SparkSession.builder.getOrCreate()

In [6]:
from datetime import datetime, date
from pyspark.sql import Row

In [7]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

In [8]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



# ZONA DE MEDIA

In [9]:
df_data = spark.createDataFrame([
    ("Jorge", 10, 8, 5, None),
    ("Fito", 6, 7, 8, 10),
    ("Percy", None, None, None, 5)
],["Nombre", "Nota1", "Nota2", "Nota3", "Nota4"])

In [10]:
df_data.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Nota1: long (nullable = true)
 |-- Nota2: long (nullable = true)
 |-- Nota3: long (nullable = true)
 |-- Nota4: long (nullable = true)



OBTENER LAS COLUMNAS NOTAS

In [11]:
#Zona de crear la lista de columnas
columnas = [x for x in df_data.columns if "Nota" in x]

In [12]:
print(columnas)

['Nota1', 'Nota2', 'Nota3', 'Nota4']


FUNCION UDF

In [13]:
@udf(returnType=FloatType())
def met_media(columnas_listas):
    suma = 0
    contador = 0
    for rows in columnas_listas:
        value = rows
        if value is not None:
            suma +=value
            contador +=1

    if suma > 0:
        return suma / contador
    else:
        return 0

In [14]:
df_media = df_data.withColumn("MET_MEDIA", met_media(array(columnas)))
df_media.show()

+------+-----+-----+-----+-----+---------+
|Nombre|Nota1|Nota2|Nota3|Nota4|MET_MEDIA|
+------+-----+-----+-----+-----+---------+
| Jorge|   10|    8|    5| null|7.6666665|
|  Fito|    6|    7|    8|   10|     7.75|
| Percy| null| null| null|    5|      5.0|
+------+-----+-----+-----+-----+---------+

